In [195]:
## 0.A Import libraries
import os

# dates
from datetime import datetime

# data handling
import numpy as np
import pandas as pd

# Data viz
%matplotlib inline
import matplotlib.pyplot as plt

# custom
from utils.custom_functions import prepare_gdp_file

In [196]:
## O.C Arguments Paths
gravity_filename = r"gravity_1995_2014.dta"

trade_filename = r"chinese_trade.csv"

gdp_filename_1 = r"gdp_1.csv"
gdp_filename_2 = r"gdp_2.csv"

population_filename = r"pop.csv"

country_code_filename = r"iso-country-codes.csv"

tweets_refactored_filename = r"groupedv2_refactored.csv"

In [197]:
folder_path = os.path.dirname(os.getcwd())
path_to_data = os.path.join(folder_path,'data')
path_to_data

'/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/gravity_trade/data'

## Load ALL

In [198]:
country_code = pd.read_csv(os.path.join(path_to_data, country_code_filename))
country_code.rename(columns = {'English short name lower case' : 'Country'}, inplace = True)
country_code = country_code[['Country', 'Alpha-3 code']]

In [199]:
mapping_years = {'2012 [YR2012]': '2012',
'2013 [YR2013]': '2013',
'2014 [YR2014]': '2014',
'2015 [YR2015]': '2015',
'2016 [YR2016]': '2016',
'2017 [YR2017]': '2017',
'2018 [YR2018]': '2018',
'2019 [YR2019]': '2019',
'2020 [YR2020]': '2020',
'2021 [YR2021]': '2021',
'2022 [YR2022]': '2022'
}

In [200]:
# Load gdp data
def prepare_gdp_file(path_to_data, gdp_filename): 
    """Refactofing file.

    Args:
        path_to_data (_type_): _description_
        gdp_filename (_type_): _description_
    """
    gdp1 = pd.read_csv(os.path.join(path_to_data, gdp_filename),
    dtype={'GDP (current US$)':int}
    )
    gdp1 = gdp1[gdp1["Series Name"] == "GDP (current US$)"]
    gdp1.rename(columns=mapping_years, inplace=True)
    gdp1 = gdp1.drop(columns=['Country Name','Series Name', 'Series Code'])
    gdp1 = gdp1.set_index('Country Code')
    gdp1 = gdp1.stack().reset_index()
    gdp1.rename(columns = {'level_1':'year', 0:'gdp'}, inplace=True)
    gdp1['Key'] = gdp1['Country Code'] + '_' + gdp1['year'].astype(str)
    gdp1 = gdp1[gdp1['gdp'] != '..']
    gdp1['gdp'] = gdp1['gdp'].astype(float) 
    
    return gdp1

In [201]:
gdp1 = prepare_gdp_file(path_to_data, gdp_filename_1)

In [202]:
# load pop data
pop = pd.read_csv(os.path.join(path_to_data, population_filename))
pop = pop[pop["Series Name"] == "Population, total"]
pop.rename(columns=mapping_years, inplace=True)
pop = pop.drop(columns=['Country Name','Series Name', 'Series Code'])
pop = pop.set_index('Country Code')
pop = pop.stack().reset_index()
pop.rename(columns = {'level_1':'year', 0:'population'}, inplace=True)
pop['Key'] = pop['Country Code'] + '_' + pop['year'].astype(str)

In [203]:
# Load Gravity data
gravity = pd.read_stata(os.path.join(path_to_data, gravity_filename))
gravity.t = gravity.t.astype(int)
gravity = gravity[gravity['i']=='CHN']
distance = gravity[['j','dist']].drop_duplicates()
distance.head()

,j,dist
82529,ABW,14155.350
82548,AFG,4180.438
82568,AGO,11769.510
82588,AIA,13563.450
82596,ALB,7686.079


In [204]:
gravity = gravity[gravity['i']=='CHN']
distance = gravity[['j','dist']].drop_duplicates()

In [205]:
# join gdp and pop and dist
data = pd.merge(gdp1, pop[['population', 'Key']], on = 'Key', how = 'outer')
data.head()

,Country Code,year,gdp,Key,population
0,AFG,2012,2.020357e+10,AFG_2012,30466479
1,AFG,2013,2.056449e+10,AFG_2013,31541209
2,AFG,2014,2.055058e+10,AFG_2014,32716210
3,AFG,2015,1.999816e+10,AFG_2015,33753499
4,AFG,2016,1.801956e+10,AFG_2016,34636207


In [206]:
data = pd.merge(data, distance, left_on = 'Country Code', right_on='j', how = 'left')
data.dropna(subset = 'Country Code', inplace = True)
data

,Country Code,year,gdp,Key,population,j,dist
0,AFG,2012,2.020357e+10,AFG_2012,30466479,AFG,4180.438
1,AFG,2013,2.056449e+10,AFG_2013,31541209,AFG,4180.438
2,AFG,2014,2.055058e+10,AFG_2014,32716210,AFG,4180.438
3,AFG,2015,1.999816e+10,AFG_2015,33753499,AFG,4180.438
4,AFG,2016,1.801956e+10,AFG_2016,34636207,AFG,4180.438
...,...,...,...,...,...,...,...
2553,ZWE,2017,1.758489e+10,ZWE_2017,14751101,ZWE,10898.120
2554,ZWE,2018,3.415607e+10,ZWE_2018,15052184,ZWE,10898.120
2555,ZWE,2019,2.183223e+10,ZWE_2019,15354608,ZWE,10898.120
2556,ZWE,2020,2.150970e+10,ZWE_2020,15669666,ZWE,10898.120


In [209]:
# Add Twitter data
tweets = pd.read_csv(os.path.join(path_to_data,tweets_refactored_filename), sep = ',')
tweets.rename(columns={'Year':'year'}, inplace =True)
tweets = tweets[['sum_pos_tweets','count_tweets','sum_political_tweets','sum_likes','sum_retweeets', 'Key']]
tweets

,sum_pos_tweets,count_tweets,sum_political_tweets,sum_likes,sum_retweeets,Key
0,2,4,4,0.0,6.0,AFG_2013
1,8,18,18,10.0,15.0,AFG_2014
2,6,11,11,1.0,0.0,AFG_2015
3,17,44,44,271.0,84.0,AFG_2016
4,13,54,54,266.0,89.0,AFG_2017
...,...,...,...,...,...,...
925,3,10,10,6.0,2.0,ZWE_2017
926,10,28,28,27.0,14.0,ZWE_2018
927,14,31,31,29.0,10.0,ZWE_2019
928,7,11,11,55.0,16.0,ZWE_2020


In [210]:
data = pd.merge(data, tweets, left_on = 'Key', right_on='Key', how = 'left')
data.to_csv('data_witout_trade.csv')

In [211]:
# load trade
trade = pd.read_csv(os.path.join(path_to_data,trade_filename), sep=';')
trade['Key'] = trade['PartnerISO'] + '_' + trade['Period'].astype(str)
trade = trade[trade['FlowDesc'] == 'Import']
trade

,RefYear,Period,ReporterISO,FlowDesc,PartnerISO,Cifvalue,PrimaryValue,Key
0,2012,2012,CHN,Import,W00,1.818199e+12,1818199227571,W00_2012
1,2012,2012,CHN,Import,AFG,5.186565e+06,5186565,AFG_2012
2,2012,2012,CHN,Import,ALB,1.427209e+08,142720886,ALB_2012
3,2012,2012,CHN,Import,DZA,2.311906e+09,2311905609,DZA_2012
4,2012,2012,CHN,Import,AND,3.240020e+05,324002,AND_2012
...,...,...,...,...,...,...,...,...
4145,2021,2021,CHN,Import,WLF,1.475400e+04,14754,WLF_2021
4146,2021,2021,CHN,Import,WSM,6.432650e+05,643265,WSM_2021
4147,2021,2021,CHN,Import,YEM,4.708126e+08,470812557,YEM_2021
4148,2021,2021,CHN,Import,ZMB,4.385251e+09,4385251435,ZMB_2021


In [212]:
data = pd.merge(trade, data, on="Key", how = 'left')

In [213]:
data.to_stata("data.dta")

/var/folders/dk/28gb2q9512d3w1nv9b59mmz40000gn/T/ipykernel_1385/3245267798.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Country Code   ->   Country_Code

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  data.to_stata("data.dta")


In [214]:
data.head()

,RefYear,Period,ReporterISO,FlowDesc,PartnerISO,Cifvalue,PrimaryValue,Key,Country Code,year,gdp,population,j,dist,sum_pos_tweets,count_tweets,sum_political_tweets,sum_likes,sum_retweeets
0,2012,2012,CHN,Import,W00,1.818199e+12,1818199227571,W00_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,2012,CHN,Import,AFG,5.186565e+06,5186565,AFG_2012,AFG,2012,2.020357e+10,30466479,AFG,4180.438,NaN,NaN,NaN,NaN,NaN
2,2012,2012,CHN,Import,ALB,1.427209e+08,142720886,ALB_2012,ALB,2012,1.231983e+10,2900401,ALB,7686.079,NaN,NaN,NaN,NaN,NaN
3,2012,2012,CHN,Import,DZA,2.311906e+09,2311905609,DZA_2012,DZA,2012,2.090590e+11,37260563,DZA,9117.676,NaN,NaN,NaN,NaN,NaN
4,2012,2012,CHN,Import,AND,3.240020e+05,324002,AND_2012,AND,2012,3.188809e+09,71013,AND,8764.593,NaN,NaN,NaN,NaN,NaN


In [215]:
tweets

,sum_pos_tweets,count_tweets,sum_political_tweets,sum_likes,sum_retweeets,Key
0,2,4,4,0.0,6.0,AFG_2013
1,8,18,18,10.0,15.0,AFG_2014
2,6,11,11,1.0,0.0,AFG_2015
3,17,44,44,271.0,84.0,AFG_2016
4,13,54,54,266.0,89.0,AFG_2017
...,...,...,...,...,...,...
925,3,10,10,6.0,2.0,ZWE_2017
926,10,28,28,27.0,14.0,ZWE_2018
927,14,31,31,29.0,10.0,ZWE_2019
928,7,11,11,55.0,16.0,ZWE_2020


In [216]:
from pandas_profiling import ProfileReport

In [217]:
data.dtypes

RefYear                   int64
Period                    int64
ReporterISO              object
FlowDesc                 object
PartnerISO               object
Cifvalue                float64
PrimaryValue              int64
Key                      object
Country Code             object
year                     object
gdp                     float64
population               object
j                        object
dist                    float64
sum_pos_tweets          float64
count_tweets            float64
sum_political_tweets    float64
sum_likes               float64
sum_retweeets           float64
dtype: object

In [218]:
data.population = pd.to_numeric(data.population)

In [219]:
# Generate the ProfileReport
profile = ProfileReport(data, title="Pandas Profiling Report", explorative=True)

 # Save the report as an HTML file
path_to_save = os.path.join('/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/gravity_trade/reports', 'final_dataset' + ".html")
profile.to_file(path_to_save)
  

Summarize dataset:  77%|███████▋  | 20/26 [00:00<00:00, 17.23it/s, Calculate auto correlation]            /Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/gravity_trade/venv_trade/lib/python3.9/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 49.10it/s]


In [32]:
data.population.dropna().describe()

count         2048
unique        2048
top       30466479
freq             1
Name: population, dtype: object

In [177]:
tweets

,sum_pos_tweets,count_tweets,sum_political_tweets,sum_likes,sum_retweeets,Key
0,2,4,4,0.0,6.0,AFG_2013.0
1,8,18,18,10.0,15.0,AFG_2014.0
2,6,11,11,1.0,0.0,AFG_2015.0
3,17,44,44,271.0,84.0,AFG_2016.0
4,13,54,54,266.0,89.0,AFG_2017.0
...,...,...,...,...,...,...
925,3,10,10,6.0,2.0,ZWE_2017.0
926,10,28,28,27.0,14.0,ZWE_2018.0
927,14,31,31,29.0,10.0,ZWE_2019.0
928,7,11,11,55.0,16.0,ZWE_2020.0


In [178]:
data.head()

,RefYear,Period,ReporterISO,FlowDesc,PartnerISO,Cifvalue,PrimaryValue,Key,Country Code,year,gdp,population,j,dist,sum_pos_tweets,count_tweets,sum_political_tweets,sum_likes,sum_retweeets
0,2012,2012,CHN,Import,W00,1.818199e+12,1818199227571,W00_2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,2012,CHN,Import,AFG,5.186565e+06,5186565,AFG_2012,AFG,2012,2.020357e+10,30466479.0,AFG,4180.438,NaN,NaN,NaN,NaN,NaN
2,2012,2012,CHN,Import,ALB,1.427209e+08,142720886,ALB_2012,ALB,2012,1.231983e+10,2900401.0,ALB,7686.079,NaN,NaN,NaN,NaN,NaN
3,2012,2012,CHN,Import,DZA,2.311906e+09,2311905609,DZA_2012,DZA,2012,2.090590e+11,37260563.0,DZA,9117.676,NaN,NaN,NaN,NaN,NaN
4,2012,2012,CHN,Import,AND,3.240020e+05,324002,AND_2012,AND,2012,3.188809e+09,71013.0,AND,8764.593,NaN,NaN,NaN,NaN,NaN
